# Inferring Hierarchical Representations

Today:
- We will be working in **groups of two or more**.
- **Choose a new partner** that you haven't worked with before.
- Move chairs together and discuss as you work!

In [3]:
# Run this cell first
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tools import *

### Hierarchies in Cognition

Hierarchical organization is a fundamental structure in human cognition. Think about how we categorize a canary. We first distinguish between animals and non-animals at the highest level of a hierarchy. Then perhaps between birds and mammals, flying and non-flying birds, and singing/non-singing birds at lower levels of the hierarchy.

Collins and Quillian (1969) showed that people's reaction time to verify semantic facts depends on the hierarchical level. For example, people were faster at verifying that "a canary can fly" than "a canary has skin". The reason is that verifying flight can be done by traversing a short path through the hierarchy from canary → bird (where the "can fly" property is stored), but verifying the property "has skin" requires moving from canary → bird → animal (where the "has skin" property is stored). Hierarchical representations facilitate efficient information processing because properties are stored only once at the most general applicable level, avoiding redundant storage at lower levels.

We previously learned how we can use similarity data to infer how people represent objects in their minds. In particular, we inferred aspects of these representations that are spatial (i.e., sets of points in a continuous space), and featural (e.g., sets of discrete features).

In this lab, we will learn how to infer a special kind of feature representation called **hierarchical feature representations**. In particular, our goal will be to build a hierarchy of features that characterize the objects in our data and that follow from the similarity data.

### Hierarchical Clustering

The **hierarchical clustering** algorithm, also commonly called **agglomerative clustering**, starts by treating each object as a separate cluster. Then, it repeatedly executes the following two steps:

1. identify the two clusters that are most similar, and
2. merge those two clusters into a single cluster.

This iterative process continues until all the clusters are merged together.

To see how this algorithm is applied, let's load some similarity data for five animals:

In [4]:
df_sim = load_small_similarity_data()
df_sim

,Dog,Wolf,Eagle,Goldfish,Dolphin
Dog,1.0,0.8,0.3,0.2,0.3
Wolf,0.8,1.0,0.3,0.2,0.2
Eagle,0.3,0.3,1.0,0.1,0.3
Goldfish,0.2,0.2,0.1,1.0,0.7
Dolphin,0.3,0.2,0.3,0.7,1.0


**Exercise:** 

Looking at the similarity matrix above, which pair of animals do people think are the most similar?

Which pair is the second most similar?

In [9]:
# replace ??? with the correct animal names

most_similar_pair = ["Dog", "Wolf"]
second_most_similar_pair = ["Dolphin", "Goldfish"]

# do not change
print("The most similar pair is:", most_similar_pair)
print("The second most similar pair is:", second_most_similar_pair)

The most similar pair is: ['Dog', 'Wolf']
The second most similar pair is: ['Dolphin', 'Goldfish']


In [10]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if (set(most_similar_pair) == {"Dog", "Wolf"} and 
    set(second_most_similar_pair) == {"Goldfish", "Dolphin"}):
    print('Test passed')
else:
    print('Test failed')

Test passed


Before grouping objects into clusters, we can think of each object (e.g., animal) as already belonging to a **cluster** (i.e., a group) that contains only that one object (e.g., the "dog" cluster contains "dog"). A cluster of only one object is called a **singleton**.

**Exercise:** Create a list called `initial_clusters` that contains each animal as its own cluster (a list with one element). The order should match that of the index of `df_sim`. This will be our starting point for hierarchical clustering.

In [11]:
# Your code here

initial_clusters = [[x] for x in df_sim.index]

In [12]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if (isinstance(initial_clusters, list) and
    len(initial_clusters) == 5 and 
    all(len(cluster) == 1 for cluster in initial_clusters) and
    # matches order of animals in df_sim
    np.all(np.array([x[0] for x in initial_clusters]) == df_sim.index)
):
    print('Test passed')
else:
    print('Test failed')

Test passed


Recall that "cluster" is another word for feature, because both pick out a subset of objects to which they apply. Thus any set of clusters is also a feature representation.

**Exercise:** Edit the values of `feature_matrix` below to construct a feature representation that matches the features you defined in `initial_clusters`.

In [15]:
# edit the values of this matrix
feature_matrix = [
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1]
]

df_initial = pd.DataFrame(
    feature_matrix, 
    index=["Dog", "Wolf", "Eagle", "Goldfish", "Dolphin"], 
    columns=["Cluster1", "Cluster2", "Cluster3", "Cluster4", "Cluster5"]
)

df_initial

,Cluster1,Cluster2,Cluster3,Cluster4,Cluster5
Dog,1,0,0,0,0
Wolf,0,1,0,0,0
Eagle,0,0,1,0,0
Goldfish,0,0,0,1,0
Dolphin,0,0,0,0,1


In [16]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if np.all(df_initial.sum(axis=0) == 1):
    print('Test passed')
else:
    print('Test failed')

Test passed


**Exercise:** What's the largest number of shared features between any two objects in the feature matrix above?

In [19]:
largest_n_shared = 0

In [20]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if largest_n_shared == 272 % 17:
    print('Test passed')
else:
    print('Test failed')

Test passed


This above is not a very interesting feature representation. For example, there are no shared features to determine similarity, and there is no hierarchy. However, this is just a starting point for additional clustering using hierarchical clustering.

The next step in hierarchical clustering is to **group the two most similar clusters** (objects).

**Exercise:** Create a function called `find_most_similar_pair` that takes a similarity matrix (DataFrame) and returns the names of the two most similar objects (the pair with the highest similarity value) as a tuple of two strings (e.g., `('animal1', 'animal2')`).

**Warm-up:** Since you've already had more than enough experience with indexing, the first cell below is a reminder of how to iterate though unique pairs of objects. There are few straightforward blanks to fill in. Once complete, you can use the code as part of your solution for your `find_most_similar_pair` function in the second cell.

In [21]:
# Just a warm up. `find_most_similar_pair` goes in next cell.

for i in range(len(df_sim)):
    for j in range(len(df_sim)):

        if i < j: # only check (dog, wolf) since (dog, wolf) == (wolf, dog)

            obj1 = df_sim.index[i] # e.g., dog
            obj2 = df_sim.index[j] # e.g., wolf

            sim = df_sim.loc[obj1, obj2] # similarity between obj1 and obj2
            print((obj1, obj2), 'have a similarity score of', sim)

# Just a warm up. `find_most_similar_pair` goes in next cell.

('Dog', 'Wolf') have a similarity score of 0.8
('Dog', 'Eagle') have a similarity score of 0.3
('Dog', 'Goldfish') have a similarity score of 0.2
('Dog', 'Dolphin') have a similarity score of 0.3
('Wolf', 'Eagle') have a similarity score of 0.3
('Wolf', 'Goldfish') have a similarity score of 0.2
('Wolf', 'Dolphin') have a similarity score of 0.2
('Eagle', 'Goldfish') have a similarity score of 0.1
('Eagle', 'Dolphin') have a similarity score of 0.3
('Goldfish', 'Dolphin') have a similarity score of 0.7


In [36]:
# Your code goes here
def find_most_similar_pair(df):
    sim = {}
    for i in range(len(df_sim)):
        for j in range(len(df_sim)):

            if i < j: # only check (dog, wolf) since (dog, wolf) == (wolf, dog)

                obj1 = df_sim.index[i] # e.g., dog
                obj2 = df_sim.index[j] # e.g., wolf

                sim[df_sim.loc[obj1, obj2]] = [obj1, obj2] # similarity between obj1 and obj2

    highest = sorted(sim.keys())[-1]
    return sim[highest]

# don't edit this code
most_similar = find_most_similar_pair(df_sim)
print(f"The most similar pair is {most_similar}")

The most similar pair is ['Dog', 'Wolf']


In [37]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if set(most_similar) == {"Dog", "Wolf"}:
    print('Test passed')
else:
    print('Test failed')

Test passed


Now that we've identified the most similar pair of objects (Dog and Wolf), we can group them together to form our first higher level cluster that contains more than one object.

A cluster can be represented as a list of objects. For example, `["Dog", "Wolf"]` is a cluster containing Dog and Wolf.

Replacing the two most similar clusters, `["Dog"]` and `["Wolf"]`), with the single cluster `["Dog", "Wolf"]` is called **merging**.

**Exercise:** Create a new list called `clusters_after_first_merge` that is like `initial_clusters` but where the individual clusters for Dog and Wolf have been replaced with a single merged cluster containing both animals. The order of the clusters should be: all *remaining* singletons in their original order first, followed by our new merged cluster.

In [42]:
# Your code here
clusters_after_first_merge = [most_similar] + initial_clusters[2:]

# do not change
print(clusters_after_first_merge)

[['Dog', 'Wolf'], ['Eagle'], ['Goldfish'], ['Dolphin']]


In [43]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if (len(clusters_after_first_merge) == 4 and  # Should be 4 clusters now
    any(set(cluster) == {"Dog", "Wolf"} for cluster in clusters_after_first_merge) and  # Contains Dog-Wolf cluster
    sum(len(cluster) == 1 for cluster in clusters_after_first_merge) == 3):  # 3 singleton clusters
    print('Test passed')
else:
    print('Test failed')

Test passed


After our first merge, we have one non-singleton (merged) cluster: `["Dog", "Wolf"]`. This cluster can be viewed as a mental feature that Dog and Wolf share, perhaps something like **Canines**.

In doing this, we've officially begun to build our **hierarchy** or **tree**. Below is a visualization of this tree. The **root** of the tree is the (current) highest-level category which we're calling "Canines". "Canines" is then split into two **branches**. When those branches immediately connect to singletons, they are called **leaves** (here, Dog and Wolf).

```
     Canines     
     /     \     
  Dog      Wolf  
```

Let's create a feature matrix using our new merged cluster. 

Note that we will leave out the original singleton features for simplicity going forward. These singleton features (representing individual animals) are less informative for understanding the hierarchical structure since each applies to only one object. By focusing only on the merged clusters, we can better visualize how the hierarchy develops through shared features that apply to multiple objects. Note that these singleton features would still exist in a complete representation but don't contribute to showing relationships between objects.

**Exercise:** Edit the values of `feature_matrix` below to construct a feature representation that matches the new non-singleton feature in `clusters_after_first_merge` that you created through merging.

In [44]:
# edit the values of this matrix
feature_matrix = [
    [1],
    [1],
    [0],
    [0],
    [0]
]

df_merge1 = pd.DataFrame(
    feature_matrix, 
    index=["Dog", "Wolf", "Eagle", "Goldfish", "Dolphin"], 
    columns=["Canines"]
)

df_merge1

,Canines
Dog,1
Wolf,1
Eagle,0
Goldfish,0
Dolphin,0


In [45]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if (df_merge1.loc["Dog", "Canines"] == 1.0 and
    df_merge1.loc["Wolf", "Canines"] == 1.0 and
    df_merge1.loc["Eagle", "Canines"] == 0.0):
    print('Test passed')
else:
    print('Test failed')

Test passed


After our first merge, our data is now partitioned into 4 distinct clusters:
- The `[Dog, Wolf]` nonsingleton cluster
- The `[Eagle]` singleton cluster
- The `[Goldfish]` singleton cluster
- The `[Dolphin]` singleton cluster

This represents a reduction from our initial 5 separate clusters to 4 clusters.

That is, we started with:

In [46]:
print(initial_clusters, '\n')

print("initial_clusters has", len(initial_clusters), "clusters")

[['Dog'], ['Wolf'], ['Eagle'], ['Goldfish'], ['Dolphin']] 

initial_clusters has 5 clusters


And, now we have:

In [47]:
print(clusters_after_first_merge, '\n')

print("clusters_after_first_merge has", len(clusters_after_first_merge), "clusters")

[['Dog', 'Wolf'], ['Eagle'], ['Goldfish'], ['Dolphin']] 

clusters_after_first_merge has 4 clusters


The hierarchical clustering process continues by repeatedly finding and merging the most similar pair of clusters until all objects are in a single cluster.

For the next iteration, we'll need to compare similarities between these 4 remaining clusters to determine which pair to merge next, just like we did before.

Now that one of the clusters is non-singleton, there are two possibilities: 
1. two more singletons (e.g., Goldfish and Dolphin) will be most similar and thus be merged, or 
2. a singleton and non-singleton (e.g., Eagle and [Dog, Wolf]) will be most similar and thus merged.

While we already know how to compare two singletons as in (1) by checking their similarity value, we don't yet know how to compare similarity between a singleton and non-singleton as in (2).

To solve this, we will define the similarity between two clusters as the **average similarity between all pairs of members**. This method is called **average linkage**. 

For pairs of singletons, there is only one member in each cluster, so the average similarity is just their similarity score as before. However, when comparing say Eagle to the [Dog, Wolf] cluster, we will average the similarity between Eagle-Dog and Eagle-Wolf.

**Exercise:** Create a function called `compute_cluster_similarity` that takes two clusters (lists of object names) and a similarity matrix, and returns the average similarity between all pairs of objects, one from each cluster.

For example, to get the similarity between the Eagle and [Dog, Wolf] clusters, you could call:

```python
    compute_cluster_similarity(["Dog", "Wolf"], ["Eagle"], df_sim)
```

This will return an average similarity value between 0 and 1 (same scale as the original values).

**HINT 1:** Recall that you can access similarity for a pair using e.g., `df_sim.loc["Dog", "Wolf"]`.

**HINT 2:** The function should also work when *both* input lists are non-singletons like `["Dog", "Wolf"]`.

In [65]:
# Your code here

def compute_cluster_similarity(cluster1, cluster2, sim_matrix):
    avg = []
    for i in cluster1:
        for j in cluster2:
            avg.append(df_sim.loc[i,j])
    return np.average(avg)

If correctly written, the function should apply to two singletons.

The output of the next two cells should match.

In [59]:
dog_wolf_sim = df_sim.loc["Dog", "Wolf"]
dog_wolf_sim

np.float64(0.8)

In [66]:
dog_wolf_cluster_sim = compute_cluster_similarity(["Dog"], ["Wolf"], df_sim)
dog_wolf_cluster_sim

np.float64(0.8)

In [67]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if np.isclose(dog_wolf_sim, dog_wolf_cluster_sim):
    print('Test passed')
else:
    print('Test failed')

Test passed


It should also apply to a singleton and non-singleton.

In [68]:
sing_to_non = compute_cluster_similarity(["Dog"], ["Eagle", "Goldfish"], df_sim)
sing_to_non

np.float64(0.25)

In [69]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if np.isclose(sing_to_non, float('52.0'[::-1])):
    print('Test passed')
else:
    print('Test failed')

Test passed


Now that we can compute similarity between clusters, let's find the next pair of clusters to merge.

**Exercise:** 

Create a function called `find_most_similar_clusters` that takes a list of clusters like `clusters_after_first_merge` and a similarity matrix, and returns the indices of the two most similar clusters as a tuple `(i, j)`.

**HINT:** You can borrow some of your code from the `find_most_similar_pair` function you already wrote above.

In [82]:
# Your code here
def find_most_similar_clusters(clusters, sim_matrix) -> tuple:
    sim = {}
    
    for i in range(len(clusters)):
        for j in range(len(clusters)):
            if i < j:
                sim[sim_matrix.iloc[i, j]] = (i, j)
            
    return sim[sorted(sim.keys())[-1]]
        
# do not change
i_, j_ = find_most_similar_clusters(clusters_after_first_merge, df_sim)
print(f"Your function thinks the most similar clusters are {clusters_after_first_merge[i_]} and {clusters_after_first_merge[j_]}")

Your function thinks the most similar clusters are ['Dog', 'Wolf'] and ['Eagle']


In [81]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
i_, j_ = find_most_similar_clusters(clusters_after_first_merge, df_sim)
cluster1 = clusters_after_first_merge[i_]
cluster2 = clusters_after_first_merge[j_]

if (set(cluster1) == {"Goldfish"} and set(cluster2) == {"Dolphin"}) or \
   (set(cluster1) == {"Dolphin"} and set(cluster2) == {"Goldfish"}):
    print('Test passed')
else:
    print('Test failed')

Test failed


Great! We've found that Goldfish and Dolphin should be our next cluster. Let's merge those clusters.

To help you merge clusters going forward, you can use a preloaded function called `merge_clusters`, which takes as input the current set of clusters and the two indices of the clusters to merge, e.g.,  `merge_clusters(my_current_clusters, i, j)`, and returns a new, smaller list of clusters with the merge.

**Exercise:** Use your existing functions along with `merge_clusters` to create a new list of clusters called `clusters_after_second_merge` where the Goldfish and Dolphin clusters are merged.

In [ ]:
# Your code here



# do not change
print("Clusters after second merge:")
print(clusters_after_second_merge)

In [ ]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if (len(clusters_after_second_merge) == 3 and  # Should be 3 clusters now
    any(set(cluster) == {"Dog", "Wolf"} for cluster in clusters_after_second_merge) and  # Contains Dog-Wolf
    any(set(cluster) == {"Goldfish", "Dolphin"} for cluster in clusters_after_second_merge) and  # Contains Goldfish-Dolphin
    any(set(cluster) == {"Eagle"} for cluster in clusters_after_second_merge)):  # Contains Eagle
    print('Test passed')
else:
    print('Test failed')

Our (incomplete) tree now looks like this.

```
     Canines               Aquatic        
     /     \             /         \      
  Dog      Wolf      Goldfish    Dolphin  
```

**Exercise:** Let's update our feature matrix to include the new cluster.

In [ ]:
# edit the values of this matrix
feature_matrix = [
    [1, 0],
    [1, 0],
    [0, 0],
    [0, 0],
    [0, 0]
]

df_merge2 = pd.DataFrame(
    feature_matrix, 
    index=["Dog", "Wolf", "Eagle", "Goldfish", "Dolphin"], 
    columns=["Canines", "Aquatic"]
)

df_merge2

In [ ]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if (df_merge2.loc["Dolphin", "Aquatic"] == 1.0 and
    df_merge2.loc["Goldfish", "Aquatic"] == 1.0 and
    df_merge2.loc["Eagle", "Aquatic"] == 0.0):
    print('Test passed')
else:
    print('Test failed')

Now we have two features - one for the Dog-Wolf cluster (perhaps "Canines") and one for the Goldfish-Dolphin cluster (perhaps "Aquatic Animals").

Let's continue the merging process.

**Exercise:** Find the most similar clusters in `clusters_after_second_merge` and merge them into `clusters_after_third_merge` using the functions you've created and that have been provided.

In [ ]:
# Your code here



# don't change
print("\nClusters after third merge:")
print(clusters_after_third_merge)

In [ ]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if (len(clusters_after_third_merge) == 2 and
    any(set(cluster) == {"Dog", "Wolf", "Eagle"} for cluster in clusters_after_third_merge) and
    any(set(cluster) == {"Goldfish", "Dolphin"} for cluster in clusters_after_third_merge)):
    print('Test passed')
else:
    print('Test failed')

We now have a new feature (maybe "Land Animals") that combines Canines with Eagle.

**Exercise:** Complete the extended feature matrix.

In [ ]:
# edit the values of this matrix
feature_matrix = [
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 0],
    [0, 1, 0],
    [0, 1, 0]
]

df_merge3 = pd.DataFrame(
    feature_matrix, 
    index=["Dog", "Wolf", "Eagle", "Goldfish", "Dolphin"], 
    columns=["Canines", "Aquatic", "Land"]
)

df_merge3

In [ ]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if (df_merge3.loc["Eagle", "Land"] == 1.0 and
    df_merge3.loc["Goldfish", "Land"] == 0.0 and
    df_merge3.loc["Dolphin", "Land"] == 0.0):
    print('Test passed')
else:
    print('Test failed')

Our partially constructed tree now looks like the below.

```
          Land         Aquatic        
         /    \       /      \        
   Canines     |      |       |       
   /     \     |      |       |       
Dog      Wolf Eagle Goldfish Dolphin  
```

Notice that the Land cluster/feature **contains** the Canines cluster/feature.

**Exercise:** If we continue merging clusters, how many more merges are possible?

In [ ]:
n_merges_left = ?

In [ ]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if 729 % 364 == n_merges_left:
    print('Test passed')
else:
    print('Test failed')

**Exercise:** Perform another merge and create `clusters_after_fourth_merge`.

In [ ]:
# Your code here



In [ ]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if (len(clusters_after_fourth_merge) == 1 and
    set(clusters_after_fourth_merge[0]) == {"Dog", "Wolf", "Eagle", "Goldfish", "Dolphin"}):
    print('Test passed')
else:
    print('Test failed')

**Exercise:** Complete the extended feature matrix.

In [ ]:
# edit the values of this matrix
feature_matrix = [
    [1, 0, 1, 0],
    [1, 0, 1, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 1, 0, 0]
]

df_merge4 = pd.DataFrame(
    feature_matrix, 
    index=["Dog", "Wolf", "Eagle", "Goldfish", "Dolphin"], 
    columns=["Canines", "Aquatic", "Land", "Animal"]
)

df_merge4

We've now arrived at a completed tree and a proper hierarchy:

```
                Animal                
               /      \               
          Land         Aquatic        
         /    \       /      \        
   Canines     |      |       |       
   /     \     |      |       |       
Dog      Wolf Eagle Goldfish Dolphin  
```

**Exercise:** How many features were discovered in the end?

In [ ]:
n_discovered = ?

In [ ]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if 730 % 363 == n_discovered:
    print('Test passed')
else:
    print('Test failed')

Let's now put everything together into a complete hierarchical clustering algorithm. We'll track both the clusters at each step and the merges that occurred.

**Exercise:** Use a single loop to implement hierarchical clustering using the functions created/provided. Save the merge history in a list of tuples called `merge_history`. Save the cluster configurations at each step in a list called `all_clusters`.

**HINT:** To copy a list of lists in python correctly, use `new_list = old_list.copy()` rather than `new_list = old_list`.

In [ ]:
# Your code here




# do not change
print("Merge history:")
for i, (cluster1, cluster2) in enumerate(merge_history):
    print(f"Step {i+1}: Merged {cluster1} and {cluster2}")

# do not change
print("\nFinal cluster:")
print(all_clusters[-1][0])
print(all_clusters)

In [ ]:
# TEST YOUR SOLUTION

# DON'T CHANGE THIS CELL
if (len(all_clusters) == 5 and
    len(merge_history) == 4 and
    len(all_clusters[-1]) == 1 and
    len(all_clusters[-1][0]) == 5):
    print('Test passed')
else:
    print('Test failed')

References:

Collins, A. M., & Quillian, M. R. (1969). Retrieval time from semantic memory. Journal of verbal learning and verbal behavior, 8(2), 240-247.